In [2]:
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import dill

In [3]:
with open('./meta_results_pf_2step_F17.pickle','rb') as f:
    results_2step_F17 = dill.load(f)
with open('./meta_results_pf_1step_F17.pickle','rb') as f:
    results_1step_F17 = dill.load(f)
with open('./meta_results_pf_1step_F14.pickle','rb') as f:
    results_1step_F14 = dill.load(f)

In [4]:
scenario = {'2step':results_2step_F17,'1step_controlled':results_1step_F17,'1step_free':results_1step_F14}

In [5]:
tray_flag = [i for i in range(1,21)]
reactive_flag = [8,9,10,11,12,14,15,16,17,18]
product_flag = ['condenser','reactive[7]','reactive[10]','reboiler']

# Create an easy-to-carry dictionary

In [7]:
obj_data = {key: [] for key in scenario}
T_data = {key: {j:[] for j in reactive_flag} for key in scenario}
P_data = {key: {j:[] for j in product_flag} for key in scenario}
R_data = {key: [] for key in scenario}
F_data = {key: {j:[] for j in tray_flag} for key in scenario}

for key in scenario:
    for it in scenario[key].keys():
        obj_data[key].append(scenario[key][it].Solution.objective['obj']['Value'])
        for j in reactive_flag:
            T_data[key][j].append(scenario[key][it].Solution.Variable['reactive[{}].T'.format(j)]['Value'])
        for j in product_flag:
            P_data[key][j].append(scenario[key][it].Solution.Variable['{}.L[P]'.format(j)]['Value'])
        R_data[key].append(scenario[key][it].Solution.Variable['condenser.L[out]']['Value']/\
                          scenario[key][it].Solution.Variable['condenser.L[P]']['Value'])
        # fot 2-step approach, first half F is fixed, so value couldn't be found in the result object
        for j in tray_flag:
            try:
                F_data[key][j].append(scenario[key][it].Solution.Variable['reactive[{}].F'.format(j)]['Value'])
            except KeyError:
                if j in reactive_flag:
                    F_data[key][j].append(1)
                else:
                    F_data[key][j].append(0)

# Plot style sheet

In [13]:
print(plt.style.available)

['seaborn-dark', 'seaborn-darkgrid', 'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic', '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright', 'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind', 'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper', 'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep']


In [26]:
style.use('seaborn')

In [27]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(9,10),sharex=True)

for key in scenario:
    ax1.plot(obj_data[key],label=key)
    ax2.plot(R_data[key],label=key)


ax1.set_title('Obj: Gasoline')
ax1.legend()

ax2.set_title('Reflux Ratio')
ax2.legend()

plt.show()